<h2>Segmenting and Clustering Neighborhoods in the city of Toronto, Canada</h2>

In [1]:
#import libraries
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


Store the link in a variable, wikipedia_link. Then, using the function get in the request library, download the Wikipedia page with wikipedia_page as argument

In [2]:
# wikipedia link that contains the postal codes
wikipedia_link = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
raw_wikipedia_page = requests.get(wikipedia_link)

Use data attribute text to extract XML to a string & assign it to page

In [3]:
page = raw_wikipedia_page.text

Extract the table from the article

In [4]:
start_point = page.find('<table class="wikitable sortable">') 
end_point = page.find("</table>") + 8 # 8 is offset for table tag
table = page[start_point:end_point]


Read the table into a dataframe

In [5]:
df = pd.read_html(table, header = 0)
df = df[0]
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Replace "Not Assigned" with NaN

In [6]:
df.replace("Not assigned", np.nan, inplace=True)
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,NaN,NaN
1,M2A,NaN,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Drop all the rows that do not have an assigned borough

In [7]:
# simply drop whole row with NaN in "Borough" column
df.dropna(subset=["Borough"], axis=0, inplace=True)

# reset index, because we dropped two rows
df.reset_index(drop=True, inplace=True)
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


Replace Nan values in the Neighbourhood column with the Borough

In [8]:
df["Neighbourhood"].replace(np.nan, df["Borough"], inplace=True)
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


Merge rows where the Postcode and Borough are the same, separating the neighbourhood name by a comma

In [9]:
# Group By Postcode and Borough and combine the Neighbourhoods on one line
df = df.groupby(['Postcode','Borough'],sort=False,as_index=False).agg(lambda x:', '.join(x))
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


Use the shape method to print the number of rows in the dataframe

In [10]:
df.shape

(103, 3)

<h3>Part II: Get coordinates for Postal Codes</h3>

In [11]:
# Read the Geospatial Coordinates into a new data frame
df_coords = pd.read_csv('Geospatial_Coordinates.csv')
df_coords.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Merge df_coords with the original dataframe based on Postal Code

In [12]:
# Rename 'Postal Code' in df_coords to 'Postcode' for merge
df_coords.rename(index=str, columns = {'Postal Code':'Postcode'}, inplace=True)
combined_df = pd.merge(df,df_coords,on='Postcode')
combined_df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


<h3>Part III: Explore the Neighbourhoods of the North York Borough</h3>

Create a new dataframe of Neighbourhoods in the North York Borough

In [13]:
northyork_data = combined_df[combined_df['Borough'] == 'North York'].reset_index(drop=True)
northyork_data.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
3,M3B,North York,Don Mills North,43.745906,-79.352188
4,M6B,North York,Glencairn,43.709577,-79.445073


Get the coordinates for North York

In [14]:
address = 'North York, ONT'

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of North York are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of North York are 43.5909589, -79.6461044.


Create a map

In [15]:
map_northyork = folium.Map(location = [latitude, longitude], zoom_start = 11)

# add markers to map
for lat, lng, borough, neighborhood in zip(northyork_data['Latitude'], northyork_data['Longitude'], northyork_data['Borough'], northyork_data['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_northyork)  
    
map_northyork

Define Foursquare Credentials and Version

In [16]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: YXSZXM2G4C2RWWI1NEP2MQTAHAIFVDCW4UTSVEVYFKIKOCV1
CLIENT_SECRET:LMJTPH1L0P0UJACXNUGXWGSUXIDK12VM01BK2HO0VWQBSQGZ


Method to get the top 25 venues in all Neighbourhoods in North York

In [17]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Retrieve the top 25 venues that are in all North York Neighbourhoods within a radius of 500 meters.

In [18]:
LIMIT = 25
radius = 500

northyork_venues = getNearbyVenues(names=northyork_data['Neighbourhood'],
                                   latitudes=northyork_data['Latitude'],
                                   longitudes=northyork_data['Longitude']
                                  )

Parkwoods
Victoria Village
Lawrence Heights, Lawrence Manor
Don Mills North
Glencairn
Flemingdon Park, Don Mills South
Hillcrest Village
Bathurst Manor, Downsview North, Wilson Heights
Fairview, Henry Farm, Oriole
Northwood Park, York University
Bayview Village
CFB Toronto, Downsview East
Silver Hills, York Mills
Downsview West
Maple Leaf Park, North Park, Upwood Park
Humber Summit
Newtonbrook, Willowdale
Downsview Central
Bedford Park, Lawrence Manor East
Emery, Humberlea
Willowdale South
Downsview Northwest
York Mills West
Willowdale West


In [19]:
# Check the size of the dat frame
print(northyork_venues.shape)
northyork_venues.head()

(184, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,KFC,43.754387,-79.333021,Fast Food Restaurant
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Parkwoods,43.753259,-79.329656,Bella Vita Catering & Private Chef Service,43.756651,-79.331524,BBQ Joint
4,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena


Number of venues returned in each neighbourhood

In [20]:
northyork_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Bathurst Manor, Downsview North, Wilson Heights",16,16,16,16,16,16
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",25,25,25,25,25,25
"CFB Toronto, Downsview East",3,3,3,3,3,3
Don Mills North,6,6,6,6,6,6
Downsview Central,4,4,4,4,4,4
Downsview Northwest,4,4,4,4,4,4
Downsview West,4,4,4,4,4,4
"Emery, Humberlea",1,1,1,1,1,1


Number of unique categories from all the returned venues

In [21]:
print('There are {} unique categories.'.format(len(northyork_venues['Venue Category'].unique())))

There are 94 unique categories.


One hot encoding to analyse each neighbourhood

In [22]:
# one hot encoding
northyork_onehot = pd.get_dummies(northyork_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
northyork_onehot['Neighborhood'] = northyork_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [northyork_onehot.columns[-1]] + list(northyork_onehot.columns[:-1])
northyork_onehot = northyork_onehot[fixed_columns]

northyork_onehot.head()

,Neighborhood,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Store,Bike Shop,Boutique,Bridal Shop,Bubble Tea Shop,Burger Joint,Bus Stop,Business Service,Butcher,Café,Candy Store,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Deli / Bodega,Department Store,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Electronics Store,Empanada Restaurant,Event Space,Fast Food Restaurant,Food & Drink Shop,Food Court,Food Truck,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,General Entertainment,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Hockey Arena,Hotel,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Korean Restaurant,Liquor Store,Lounge,Massage Studio,Mediterranean Restaurant,Metro Station,Miscellaneous Shop,Movie Theater,Park,Pet Store,Pharmacy,Pizza Place,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Restaurant,Salon / Barbershop,Sandwich Place,Shopping Mall,Smoothie Shop,Sporting Goods Shop,Steakhouse,Storage Facility,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Store,Vietnamese Restaurant
0,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Parkwoods,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Display the dataframe size

In [23]:
northyork_onehot.shape

(184, 95)

Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [24]:
northyork_grouped = northyork_onehot.groupby('Neighborhood').mean().reset_index()
northyork_grouped

,Neighborhood,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Store,Bike Shop,Boutique,Bridal Shop,Bubble Tea Shop,Burger Joint,Bus Stop,Business Service,Butcher,Café,Candy Store,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Deli / Bodega,Department Store,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Electronics Store,Empanada Restaurant,Event Space,Fast Food Restaurant,Food & Drink Shop,Food Court,Food Truck,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,General Entertainment,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Hockey Arena,Hotel,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Korean Restaurant,Liquor Store,Lounge,Massage Studio,Mediterranean Restaurant,Metro Station,Miscellaneous Shop,Movie Theater,Park,Pet Store,Pharmacy,Pizza Place,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Restaurant,Salon / Barbershop,Sandwich Place,Shopping Mall,Smoothie Shop,Sporting Goods Shop,Steakhouse,Storage Facility,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Store,Vietnamese Restaurant
0,"Bathurst Manor, Downsview North, Wilson Heights",0.0,0.000000,0.00,0.00,0.000000,0.00,0.00,0.00,0.062500,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.0625,0.00,0.00,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.125000,0.00,0.00,0.00,0.0625,0.00,0.000000,0.0625,0.00,0.0,0.00,0.0,0.0,0.062500,0.00,0.00,0.00,0.0625,0.0625,0.0,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.000000,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.000000,0.00,0.0625,0.0625,0.00,0.000000,0.00,0.00,0.00,0.062500,0.00,0.062500,0.0625,0.00,0.000000,0.00,0.000000,0.0625,0.00,0.00,0.00,0.00,0.0625,0.00
1,Bayview Village,0.0,0.000000,0.00,0.00,0.000000,0.00,0.00,0.00,0.250000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.0000,0.00,0.00,0.000000,0.00,0.00,0.250000,0.00,0.000000,0.250000,0.000000,0.000000,0.00,0.00,0.00,0.0000,0.00,0.000000,0.0000,0.00,0.0,0.00,0.0,0.0,0.000000,0.00,0.00,0.00,0.0000,0.0000,0.0,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.250000,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.000000,0.00,0.0000,0.0000,0.00,0.000000,0.00,0.00,0.00,0.000000,0.00,0.000000,0.0000,0.00,0.000000,0.00,0.000000,0.0000,0.00,0.00,0.00,0.00,0.0000,0.00
2,"Bedford Park, Lawrence Manor East",0.0,0.000000,0.04,0.00,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.0000,0.00,0.00,0.000000,0.00,0.04,0.040000,0.00,0.000000,0.000000,0.000000,0.080000,0.04,0.00,0.04,0.0000,0.00,0.000000,0.0000,0.00,0.0,0.00,0.0,0.0,0.080000,0.00,0.00,0.00,0.0000,0.0000,0.0,0.000000,0.0,0.04,0.040000,0.000000,0.000000,0.04,0.00,0.00,0.04,0.04,0.00,0.00,0.080000,0.000000,0.04,0.00,0.04,0.00,0.0,0.0,0.0,0.0,0.00,0.000000,0.00,0.0400,0.0400,0.00,0.000000,0.00,0.04,0.00,0.040000,0.00,0.040000,0.0000,0.00,0.000000,0.00,0.000000,0.0400,0.00,0.04,0.00,0.00,0.0000,0.00
3,"CFB Toronto, Downsview East",0.0,0.333333,0.00,0.00,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.0000,0.00,0.00,0.333333,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.0000,0.00,0.000000,0.0000,0.00,0.0,0.00,0.0,0.0,0.000000,0.00,0.00,0.00,0.0000,0.0000,0.0,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.000000,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.333333,0.00,0.0000,0.0000,0.00,0.000000,0.00,0.00,0.00,0.000000,0.00,0.000000,0.0000,0.00,0.000000,0.00,0.000000,0.0000,0.00,0.00,0.00,0.00,0.0000,0.00
4,Don Mills North,0.0,0.000000,0.00,0.00,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.166667,0.00,0.000000,0.000000,0.0,0.0000,0.00,0.00,0.000000,0.00,0.0

Show the new size

In [25]:
northyork_grouped.shape

(22, 95)

Print each neighbourhood along with top 5 most common venues

In [26]:
num_top_venues = 5

for hood in northyork_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = northyork_grouped[northyork_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bathurst Manor, Downsview North, Wilson Heights----
                 venue  freq
0          Coffee Shop  0.12
1          Bridal Shop  0.06
2  Fried Chicken Joint  0.06
3                Diner  0.06
4             Pharmacy  0.06


----Bayview Village----
                 venue  freq
0   Chinese Restaurant  0.25
1                 Bank  0.25
2                 Café  0.25
3  Japanese Restaurant  0.25
4         Hockey Arena  0.00


----Bedford Park, Lawrence Manor East----
                  venue  freq
0           Coffee Shop  0.08
1  Fast Food Restaurant  0.08
2    Italian Restaurant  0.08
3              Pharmacy  0.04
4        Sandwich Place  0.04


----CFB Toronto, Downsview East----
                venue  freq
0             Airport  0.33
1                Park  0.33
2            Bus Stop  0.33
3  Miscellaneous Shop  0.00
4       Metro Station  0.00


----Don Mills North----
                  venue  freq
0  Gym / Fitness Center  0.17
1  Caribbean Restaurant  0.17
2                  Café 

Create function to sort venues in descending order

In [27]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Create new dataframe and display top 10 venues for each neighborhood

In [28]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = northyork_grouped['Neighborhood']

for ind in np.arange(northyork_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(northyork_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bathurst Manor, Downsview North, Wilson Heights",Coffee Shop,Fried Chicken Joint,Bank,Bridal Shop,Pharmacy,Pizza Place,Deli / Bodega,Diner,Sandwich Place,Shopping Mall
1,Bayview Village,Café,Bank,Japanese Restaurant,Chinese Restaurant,Vietnamese Restaurant,Diner,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop
2,"Bedford Park, Lawrence Manor East",Fast Food Restaurant,Coffee Shop,Italian Restaurant,Greek Restaurant,Liquor Store,Café,Butcher,Indian Restaurant,Ice Cream Shop,Comfort Food Restaurant
3,"CFB Toronto, Downsview East",Park,Airport,Bus Stop,Diner,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Deli / Bodega
4,Don Mills North,Caribbean Restaurant,Café,Gym / Fitness Center,Pool,Baseball Field,Japanese Restaurant,Dim Sum Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant


Run k-means to cluster into 5 clusters

In [29]:
# set number of clusters
kclusters = 5

northyork_grouped_clustering = northyork_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(northyork_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 3, 0, 4, 3, 0, 0, 3, 2, 0], dtype=int32)

Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [30]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

northyork_merged = northyork_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
northyork_merged = northyork_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

northyork_merged.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,0.0,Park,Food & Drink Shop,Fast Food Restaurant,BBQ Joint,Dim Sum Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping
1,M4A,North York,Victoria Village,43.725882,-79.315572,0.0,Intersection,Coffee Shop,Portuguese Restaurant,Hockey Arena,Vietnamese Restaurant,Clothing Store,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Deli / Bodega
2,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763,0.0,Clothing Store,Vietnamese Restaurant,Sporting Goods Shop,Boutique,Coffee Shop,Event Space,Furniture / Home Store,Miscellaneous Shop,Accessories Store,Steakhouse
3,M3B,North York,Don Mills North,43.745906,-79.352188,3.0,Caribbean Restaurant,Café,Gym / Fitness Center,Pool,Baseball Field,Japanese Restaurant,Dim Sum Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant
4,M6B,North York,Glencairn,43.709577,-79.445073,4.0,Park,Pizza Place,Japanese Restaurant,Pub,Vietnamese Restaurant,Candy Store,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant


In [31]:
northyork_merged['Cluster Labels'].replace(np.nan,0,inplace=True)

Create map to visualize clusters

In [32]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(northyork_merged['Latitude'], northyork_merged['Longitude'], northyork_merged['Neighbourhood'], northyork_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters